In [5]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv("mobiles.csv")
df.head()

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
1,Small,64,4,2,1,2815,4.5,244,57149,0.04,1.39
2,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
3,Medium,64,3,1,1,2942,4.6,5366,42999,0.10,23.07
4,Medium,128,4,2,1,2815,4.6,745,69149,0.02,5.15


# Q1 
이상치를 뽑아서 성능지표를 계산하고 평균을 구하라.

In [7]:
df1 = df.copy()
df1["sales"].std()

58.39958785566842

## <span style = "color:red"> 이상치 계산할때 표준편차*2 + 평균을 넘는것을 착가하지 마라 !!!! </span>

In [84]:
df11 = df1.loc[df1["sales"] >  (df1["sales"].mean() + df1["sales"].std()*2), :]
df11.shape

(16, 11)

In [85]:
df11["성능지표"] = df11["ROM"]/32 + df11["RAM"]/2 + df11["num_rear_camera"] + df11["num_front_camera"] + df11["battery_capacity"]/1000
df11.head()

C:\Users\younl\AppData\Local\Temp/ipykernel_21368/291321255.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11["성능지표"] = df11["ROM"]/32 + df11["RAM"]/2 + df11["num_rear_camera"] + df11["num_front_camera"] + df11["battery_capacity"]/1000


,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales,성능지표
98,Medium,128,6,2,1,4000,4.6,122001,18999,0.09,231.79,14.0
110,Large,128,6,4,2,4500,4.5,267028,15999,0.20,427.22,17.5
158,Large,32,2,2,1,5000,4.4,223672,7499,0.16,167.73,10.0
159,Large,32,2,2,1,5000,4.4,223672,7499,0.16,167.73,10.0
193,Medium,16,2,2,1,4000,4.4,226996,6499,0.07,147.52,8.5


In [86]:
round(df11["성능지표"].mean(),2)

11.01

# Q2

## <span style = "color:red"> 후면카메라 1개 제외 하라고 했는데 문제 잘 읽어보고 해라 </span>

In [14]:
df2 = df.copy()
df2_list = ["battery_capacity","ratings", "num_of_ratings","sales_price","discount_percent","sales"]

In [89]:
df2 = df2.loc[df["num_rear_camera"]!=1,]

In [114]:
df2[df2_list].corr().abs().reset_index().replace(1, 0).max()

index               sales_price
battery_capacity       0.503019
ratings                0.424129
num_of_ratings         0.949114
sales_price            0.503019
discount_percent       0.257373
sales                  0.949114
dtype: object

In [91]:
df2_corr_list = pd.Series([0.424129, 0.034902, 0.503019,0.257373,0.025680])
round(df2_corr_list.max(), 2)

0.5

## sales 는 0.949114 이다. 

# 팁 : corr에서 원하는 값을 뽑는 방법... reset_index  및 melt를 활용할수 있다.

In [99]:
df_corr_melt = df2[df2_list].corr().abs().reset_index().melt(id_vars = "index").head()

In [104]:
df_corr_melt_sub = df_corr_melt.loc[df_corr_melt["index"] != df_corr_melt["variable"]]
df_corr_melt_sub["value"].max()

0.5030192095966874

# Q3

In [22]:
df3 = df.copy()
df3.columns

Index(['screen_size', 'ROM', 'RAM', 'num_rear_camera', 'num_front_camera',
       'battery_capacity', 'ratings', 'num_of_ratings', 'sales_price',
       'discount_percent', 'sales'],
      dtype='object')

In [25]:
df33 = pd.get_dummies(df3, columns=["screen_size"])

In [42]:
df33.columns

Index(['ROM', 'RAM', 'num_rear_camera', 'num_front_camera', 'battery_capacity',
       'ratings', 'num_of_ratings', 'sales_price', 'discount_percent', 'sales',
       'screen_size_Large', 'screen_size_Medium', 'screen_size_Small',
       'screen_size_Very Large', 'screen_size_Very Small'],
      dtype='object')

In [105]:
df33.head(1)

,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,64,2,1,1,1800,4.5,38645,32999,0.17,127.52,0,0,0,0,1


# columns 이름중에 빈칸이 있을때 _ 로 바꿔주는 방법
``` python
df3.columns.str.replace(" ", "_")
```

In [44]:
indep_list = ['ROM', 'RAM', 'num_rear_camera', 'num_front_camera', 'battery_capacity',
       'ratings', 'num_of_ratings', 'sales_price', 'discount_percent', 'sales',
       'screen_size_Large', 'screen_size_Medium', 'screen_size_Small',
       'screen_size_Very Large', 'screen_size_Very Small']

dep_list = ['sales']

# 특정 columns값 앞으로 옮기기
df3 = df3.set_index("sales").reset_index()

In [30]:
train, test = train_test_split(df33, random_state=123, train_size= 0.8)

In [31]:
len(train), len(test)

(344, 86)

In [43]:
train.columns

Index(['ROM', 'RAM', 'num_rear_camera', 'num_front_camera', 'battery_capacity',
       'ratings', 'num_of_ratings', 'sales_price', 'discount_percent', 'sales',
       'screen_size_Large', 'screen_size_Medium', 'screen_size_Small',
       'screen_size_Very Large', 'screen_size_Very Small'],
      dtype='object')

In [32]:
MM = MinMaxScaler().fit(train)

In [46]:
train_m = pd.DataFrame(MM.transform(train), columns= train.columns)
test_m = pd.DataFrame(MM.transform(test),  columns= test.columns)

In [116]:
from tqdm.notebook import tqdm

n_list = [3,5,7,9,11]
ls_rmse = []  ## 추가

min_v = 9999
min_c = 0

for n in tqdm(n_list):
    model = KNeighborsRegressor(n_neighbors=n).fit(X = train_m[indep_list], y = train_m[dep_list])
    pred = model.predict(test_m[indep_list])
    acc = mean_squared_error(y_true=test_m[dep_list], y_pred=pred)**0.5
    ls_rmse  = ls_rmse +[acc]
    
    print(n, acc)
    if(min_v > acc):
        min_v = acc
        min_c = n

display(min_c, min_v)

  0%|          | 0/5 [00:00<?, ?it/s]

3 0.05815373703785327
5 0.07703665678443909
7 0.09124204891260387
9 0.0999585824713245
11 0.10533127243401268


3

0.05815373703785327

In [123]:
best  = pd.Series(ls_rmse, index=n_list).idxmin()
best

3

3개..

# Q4 MinMaxScale 할때 순서가 중요하다.

In [ ]:
'ROM', 'RAM', 'num_rear_camera', 'num_front_camera', 'battery_capacity',
       'ratings', 'num_of_ratings', 'sales_price', 'discount_percent', 
       'screen_size_Large', 'screen_size_Medium', 'screen_size_Small',
       'screen_size_Very Large', 'screen_size_Very Small', 'sales'

# Tip... 기존에 있는 DF을 활용해서 만든다. ㅜㅜ
df_s1 = test.head(1).reset_index(drop=True)
df_s1

In [64]:
df4 = pd.DataFrame(np.array([[256, 6, 4, 1, 4000, 4.3, 25000, 85000, 0.05, 1, 0, 0, 0, 0, 0]]),
                    columns=['a1', 'a2', 'a3', 'a4', 'a5', 'a6','a7', 'a8', 'a9', 'a10',
                             'a11', 'a12', 'a13', 'a14', 'a15'])

In [66]:
df4.columns = ['ROM', 'RAM', 'num_rear_camera', 'num_front_camera', 'battery_capacity',
       'ratings', 'num_of_ratings', 'sales_price', 'discount_percent', 
       'screen_size_Large', 'screen_size_Medium', 'screen_size_Small',
       'screen_size_Very Large', 'screen_size_Very Small', 'sales']

In [67]:
df4

,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small,sales
0,256.0,6.0,4.0,1.0,4000.0,4.3,25000.0,85000.0,0.05,1.0,0.0,0.0,0.0,0.0,0.0


In [143]:
df4_m = pd.DataFrame(MM.transform(df4), columns= df4.columns)

In [144]:
df4_m

,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small,sales
0,0.492063,0.454545,1.0,0.0,0.423077,0.625,0.053081,0.518158,0.093023,0.002024,0.0,0.0,0.0,0.0,0.0


In [145]:
    model = KNeighborsRegressor(n_neighbors=3).fit(X = train_m[indep_list], y = train_m[dep_list])
    pred = model.predict(df4_m[indep_list])

In [146]:
pred

array([[0.00132259]])

In [147]:
MM.data_min_

array([8.000e+00, 1.000e+00, 1.000e+00, 1.000e+00, 1.800e+03, 3.800e+00,
       4.000e+00, 6.499e+03, 1.000e-02, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00])

In [148]:
MM.data_max_

array([5.12000e+02, 1.20000e+01, 4.00000e+00, 3.00000e+00, 7.00000e+03,
       4.60000e+00, 4.70905e+05, 1.57999e+05, 4.40000e-01, 4.93980e+02,
       1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00, 1.00000e+00])

In [127]:
min_v = 0
max_v = 4.93980e+02

# 0.00132259 = x - min_v / max_v - min_v
x = 0.00132259 * (max_v - min_v) + min_v
print(x)

0.6533330082000001


# 역변환

In [149]:
df4_m["sales"]=pred

In [150]:
MM.inverse_transform(df4_m)

array([[2.56000000e+02, 6.00000000e+00, 4.00000000e+00, 1.00000000e+00,
        4.00000000e+03, 4.30000000e+00, 2.50000000e+04, 8.50000000e+04,
        5.00000000e-02, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.32259066e-03]])

In [151]:
pd.DataFrame(df4_m, columns = df4.columns)

,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small,sales
0,0.492063,0.454545,1.0,0.0,0.423077,0.625,0.053081,0.518158,0.093023,0.002024,0.0,0.0,0.0,0.0,0.001323
